# Opertura d'una nova oficina i predicció de potencials localitzacions futures
 

|   INTEGRANTS                    | Arnau Muñoz Barrera     | Míriam López Sánchez       | Luis Martínez Zamora       | Marc Rodríguez Vitolo
| -------------------------- | ----------------------- | -------------------------- | -------------------------- | --------------------------

### 1. Objectius

L'objectiu d'aquest projecte es centra en buscar ubicacions idònies per obrir noves oficines de la Caixa d'Enginyers, valorant tant la possiblitat d'oficines fixes com oficinse mòbils que arriben a una determinada zona. A més, es desenvoluparà també un model predictiu amb l'ajuda d'IA que permeti identificar potencials localitzacions futures.

### 2. Font de les dades

Com a dades per analitzar en aquest projecte s'han agafat conjunts de dades extrets de fonts oficials: INE, BdE i propis de Caixa d'Enginyers.

### 3. Metodologia seguida

- Recerca de dades
- Anàlisi de dades amb python (realització de gràfiques i extracció d'estadístiques)
- Interpretació de resultats
- Elecció de les millors ubicacions posibles
- Entrenament del model predictiu
- Test del model predictiu
- Extracció de conclusions


### 4. Anàlisi de les dades

Com a primer pas, caldrà que importar totes les llibreries i utilitats que es faran servir en el desenvolupament del projecte:


In [ ]:
%pip install pandas seaborn scikit-learn category_encoders scikit-optimize scipy openpyxl

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.cm import gist_heat_r
import pandas as pd
import seaborn as sns
from matplotlib.ticker import FuncFormatter
import matplotlib.cm as cm

import warnings
warnings.simplefilter(action='ignore')

A continuació, es farà la importació dels datasets (arxius d'extensió **.xlsx**) a analitzar i s'analitzaran breument:

In [ ]:
dataframe_bancs_per_provincia = pd.read_excel('data/Bancs per provincia.xlsx', sheet_name=0, header=0, engine='openpyxl')
print("Estructura Interna de dataframe_bancs_per_provincia: ", dataframe_bancs_per_provincia.shape)

print(dataframe_bancs_per_provincia.head(5))

dfb = dataframe_bancs_per_provincia.copy()

if any(str(c).lower().startswith('columna') for c in dfb.columns):
    dfb.columns = dfb.iloc[0].tolist()
    dfb = dfb.iloc[1:].reset_index(drop=True)

dfb.columns = [str(c).strip() for c in dfb.columns]

name_col = 'Provincia'
cols_val = [
    'Banco de España',
    'Oficinas en España',
    'Entidades de depósito',
    'Otras entidades de crédito y EFC'
]

for c in cols_val:
    if c in dfb.columns:
        dfb[c] = pd.to_numeric(dfb[c], errors='coerce').fillna(0)

posibles_sel = ['Seleccion','Selección','SELECCION','SELECCIÓN','Plot','PLOT','Marcar','Include','Selected']
sel_col = next((c for c in dfb.columns if str(c).strip() in posibles_sel), None)

if sel_col is not None:
    def _to_bool(v):
        s = str(v).strip().lower()
        return (v is True) or (s in ('1','si','sí','true','x','y','yes'))
    dfb_plot = dfb[dfb[sel_col].apply(_to_bool)].copy()
else:
    if 'Oficinas en España' in dfb.columns:
        dfb_plot = dfb.sort_values('Oficinas en España', ascending=False).head(12).copy()
    else:
        dfb_plot = dfb.copy()

if 'Oficinas en España' in dfb_plot.columns:
    plt.figure(figsize=(10,6))
    plt.barh(dfb_plot[name_col], dfb_plot['Oficinas en España'])
    plt.gca().invert_yaxis()
    plt.title('Oficines bancàries per provincia')
    plt.xlabel("Nombre d'oficines")
    plt.tight_layout()
    plt.show()

presentes = [c for c in ['Banco de España','Entidades de depósito','Otras entidades de crédito y EFC'] if c in dfb_plot.columns]
if presentes:
    plt.figure(figsize=(11,6))
    bottom = np.zeros(len(dfb_plot))
    for c in presentes:
        plt.bar(dfb_plot[name_col], dfb_plot[c], bottom=bottom, label=c)
        bottom += dfb_plot[c].values
    plt.xticks(rotation=60, ha='right')
    plt.title('Entidades por provincia (desglose)')
    plt.ylabel('Número de entidades')
    plt.legend()
    plt.tight_layout()
    plt.show()

##### Punts clau
Arreu del territori espanyol es troben distribuïdes més de 17 000 oficines bancàries, sumant les de les diferents entitats que operen al país. Es pot veure que les tres provincies amb més volum d'oficines són Madrid, Barcelona i València.

In [ ]:
dataframe_empresa_per_mida_and_provincia = pd.read_excel('data/Empresa per mida y provincia 2008-2022.xlsx', sheet_name=0, header=0, engine='openpyxl')
print("Estructura Interna de dataframe_empresa_per_mida_and_provincia: ", dataframe_empresa_per_mida_and_provincia.shape)

print(dataframe_empresa_per_mida_and_provincia.head(5))

df = dataframe_empresa_per_mida_and_provincia.copy()

prov_col = 'Provincias' if 'Provincias' in df.columns else 'Provincia'

df = df[
    (df['CCAA'].str.strip().str.lower() != 'total') &      
    (df[prov_col].astype(str).str.strip().str.lower() == 'total')
]

columnas_total = [col for col in df.columns if ('Total' in col and col.split()[0].isdigit())]
columnas_total = sorted(columnas_total, key=lambda c: int(c.split()[0]))

df_grouped = df.groupby('CCAA', as_index=False)[columnas_total].sum()

plt.figure(figsize=(14, 7))

comunidades = df_grouped['CCAA'].values
num_comunidades = len(comunidades)
colors = cm.get_cmap('tab20', num_comunidades)

años = [int(col.split()[0]) for col in columnas_total]

for i, row in df_grouped.iterrows():
    comunidad = row['CCAA']
    valores = row[columnas_total].values.astype(float)
    plt.plot(años, valores, marker='o', color=colors(i), label=comunidad, linewidth=2)

plt.xlabel('Any', fontsize=12)
plt.ylabel("Nombre d'empreses", fontsize=12)
plt.title("Evolució del nombre d'empreses per comunitat autònoma", fontsize=14)
plt.gca().yaxis.set_major_formatter(FuncFormatter(lambda x, _: f'{int(x):,}'.replace(',', '.')))
plt.legend(title='Comunitat Autònoma', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True)
plt.tight_layout()
plt.show()

##### Punts clau
Es pot observar en el gràfic que la comunitat autònoma amb més empreses és Catalunya, seguida de prop per Andalucía i Madrid. Es pot destacar també la quantitat d'empreses situades a la Comunitat Valenciana.

In [ ]:
df = dataframe_empresa_per_mida_and_provincia.copy()

prov_col = 'Provincias' if 'Provincias' in df.columns else 'Provincia'

provincias_catalunya = ['Barcelona', 'Girona', 'Lleida', 'Tarragona']
df_cat = df[df[prov_col].isin(provincias_catalunya)]

columnas_total = [col for col in df_cat.columns if ('Total' in col and col.split()[0].isdigit())]
columnas_total = sorted(columnas_total, key=lambda c: int(c.split()[0]))
años = [int(col.split()[0]) for col in columnas_total]

plt.figure(figsize=(12, 6))

colors = cm.get_cmap('tab10', len(provincias_catalunya))

for i, prov in enumerate(provincias_catalunya):
    datos_prov = df_cat[df_cat[prov_col] == prov]
    valores = datos_prov[columnas_total].values.flatten().astype(float)
    plt.plot(años, valores, marker='o', linewidth=2, color=colors(i), label=prov)

plt.xlabel('Any', fontsize=12)
plt.ylabel("Nombre d'empreses", fontsize=12)
plt.title("Evolució del nombre d'empreses a Catalunya per provincies", fontsize=14)

plt.gca().yaxis.set_major_formatter(FuncFormatter(lambda x, _: f'{int(x):,}'.replace(',', '.')))

plt.legend(title='Provincia')
plt.grid(True)
plt.tight_layout()
plt.show()

##### Punts claus
Tal i com es podria suposar, la provincia de Barcelona té la major concentració d'empreses, trobant-se al voltant de 450 000. Les altres tres provincies queden notablement per sota de les 100 000.

In [ ]:
dataframe_variacio_PIB_per_capita = pd.read_excel('data/Variació PIB per capita.xlsx', sheet_name=0, header=0, engine='openpyxl')
print("Estructura Interna de dataframe_variacio_PIB_per_capita: ", dataframe_variacio_PIB_per_capita.shape)

print(dataframe_variacio_PIB_per_capita.head(5))

df = dataframe_variacio_PIB_per_capita.copy()

year_cols = [c for c in df.columns
             if str(c).isdigit() and len(str(c)) == 4 and 2000 <= int(c) <= 2099]

name_col = df.columns[0]
type_col = df.columns[1]

posibles_sel = [
    'Plot','PLOT','Selección','SELECCIÓN','Seleccion','SELECCION',
    'Marcar','MARCAR','Include','INCLUDE','Selected','SELECTED',
    'Graficar','GRAFICAR','Mostrar','MOSTRAR'
]
sel_col = next((c for c in df.columns if str(c).strip() in posibles_sel), None)

ccaa = df[df[type_col].astype(str).str.upper().str.contains('CCAA')].copy()

excluir = {'ESPAÑA', 'TOTAL', 'TOTAL NACIONAL'}
ccaa = ccaa[~ccaa[name_col].astype(str).str.upper().isin(excluir)]

if sel_col is not None:
    def _to_bool(v):
        s = str(v).strip().lower()
        return (v is True) or (s in ('1','si','sí','true','x','y','yes'))
    ccaa = ccaa[ccaa[sel_col].apply(_to_bool)]

melted = ccaa.melt(
    id_vars=[name_col, type_col],
    value_vars=year_cols,
    var_name='Año',
    value_name='PIB_per_capita'
)
melted['Año'] = melted['Año'].astype(int)

plt.figure(figsize=(11,6))
for region, sub in melted.groupby(name_col):
    plt.plot(sub['Año'], sub['PIB_per_capita'], label=region)

plt.title('Variació del PIB per càpita per comunitat autònoma')
plt.xlabel('Any')
plt.ylabel('PIB per càpita')
plt.legend(loc='best', ncol=2)
plt.tight_layout()
plt.show()



In [ ]:
dataframe_variacio_PIB_percentatge = pd.read_excel('data/Variació PIB.xlsx', sheet_name=0, header=0, engine='openpyxl')
print("Estructura Interna de dataframe_variacio_PIB_percentatge: ", dataframe_variacio_PIB_percentatge.shape)

print(dataframe_variacio_PIB_percentatge.head(5))

df2 = dataframe_variacio_PIB_percentatge.copy()

year_cols = [c for c in df2.columns
             if str(c).isdigit() and len(str(c)) == 4 and 2000 <= int(c) <= 2099]

name_col = df2.columns[0]
type_col = df2.columns[1]

posibles_sel = [
    'Plot','PLOT','Selección','SELECCIÓN','Seleccion','SELECCION',
    'Marcar','MARCAR','Include','INCLUDE','Selected','SELECTED',
    'Graficar','GRAFICAR','Mostrar','MOSTRAR'
]
sel_col = next((c for c in df2.columns if str(c).strip() in posibles_sel), None)

ccaa2 = df2[df2[type_col].astype(str).str.upper().str.contains('CCAA')].copy()

excluir = {'ESPAÑA', 'TOTAL', 'TOTAL NACIONAL'}
ccaa2 = ccaa2[~ccaa2[name_col].astype(str).str.upper().isin(excluir)]

if sel_col is not None:
    def _to_bool(v):
        s = str(v).strip().lower()
        return (v is True) or (s in ('1','si','sí','true','x','y','yes'))
    ccaa2 = ccaa2[ccaa2[sel_col].apply(_to_bool)]

melted2 = ccaa2.melt(
    id_vars=[name_col, type_col],
    value_vars=year_cols,
    var_name='Año',
    value_name='PIB'
)
melted2['Año'] = melted2['Año'].astype(int)

melted2['PIB'] = pd.to_numeric(melted2['PIB'], errors='coerce')

plt.figure(figsize=(11,6))
for region, sub in melted2.groupby(name_col):
    plt.plot(sub['Año'], sub['PIB'], label=region)

plt.title('Variació del PIB per comunitat autònoma')
plt.xlabel('Any')
plt.ylabel('PIB')
plt.legend(loc='best', ncol=2)
plt.tight_layout()
plt.show()

##### Punts clau
De les dades del PIB, la informació que s'extreu no és tant significativa, ja que en totes les provincies evoluciona de manera similar. Es pot destacar, però, que Catalunya es troba en la quarta posició del PIB per càpita més elevat d'Espanya.

### 5. Conclusions anàlisi de dades

Donats els gràfics i les sèries disponibles sobre factors clau a Espanya —com ara el PIB i el creixement de la població, que mostren una distribució desigual—, es conclou que el focus d’anàlisi amb més potencial es troba a Catalunya. Es tracta d’una de les zones més poblades, amb dinàmica de creixement i una elevada concentració d’entitats rellevants per al PIB.

A més, segons dades internes de la Caixa d'Enginyers, la distribució de socis per regió és la següent:

**⚠️ WARNING:** Distribució ficticia de socis per regions (Temporal)

| Regions              | Socis | % sobre total | Variació interanual | Nota breu                                         |
|---------------------------------|-------:|--------------:|---------------------:|----------------------------------------------------|
| Àrea Metropolitana de Barcelona | 72.000 | 60,0%         | +3,2%                | Alta concentració urbana i hubs tecnològics      |
| Comarques Gironines            | 12.600 | 10,5%         | +4,1%                | Creixement en renovables i turisme de qualitat     |
| Camp de Tarragona              | 10.800 | 9,0%          | +3,7%                | Indústria química i logística portuaria            |
| Penedès                        | 8.400  | 7,0%          | +3,5%                | Agroalimentari i teixit exportador                |
| Comarques Centrals             | 7.200  | 6,0%          | +2,8%                | Pymes industrials diversificades                  |
| Ponent                         | 4.800  | 4,0%          | +2,4%                | Agroindústria i energia                            |
| Terres de l’Ebre               | 2.400  | 2,0%          | +2,0%                | Transició energètica i serveis                  |
| Alt Pirineu i Aran             | 1.800  | 1,5%          | +1,7%                | Turisme sostenible i economia de montanya           |
| **Total**                      | **120.000** | **100%**   | —                    | —                                                  |

